In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re


def _get_soup(url: str):
    # URLからHTMLを取得
    response = requests.get(url)
    response.encoding = response.apparent_encoding
    get_soup = BeautifulSoup(response.text, "html.parser")

    return get_soup


def clean_text(text):
    if not text:
        return ""
    # 改行と余分な空白を整理
    text = re.sub(r"\s+", " ", text.strip())
    return text


def scrape_movie_info(url):
    soup = _get_soup(url)
    movies_data = []

    # 全ての.box要素を取得
    movie_boxes = soup.find_all("div", class_="box")

    for box in movie_boxes:
        try:
            # 通常の映画タイトルと特集タイトルの両方を確認
            title_elem = box.find("span", class_="eiga-title")
            title_s_elem = box.find("span", class_="title-s")

            # タイトルが見つからない場合はスキップ
            if not title_elem and not title_s_elem:
                continue

            title = clean_text(title_elem.text if title_elem else title_s_elem.text)

            # 特集作品の場合は特別な処理
            if title_s_elem:
                # 特集内の作品情報を取得
                details_elem = box.find("details")
                if details_elem:
                    sub_movies = details_elem.find_all("div", class_="box")
                    for sub_movie in sub_movies:
                        sub_title_elem = sub_movie.find("span", class_="eiga-title")
                        if not sub_title_elem:
                            continue

                        movie_info = process_movie_box(
                            sub_movie, is_special=True, special_title=title
                        )
                        if movie_info:
                            movies_data.append(movie_info)
                else:
                    movie_info = process_movie_box(box, is_special=True)
                    if movie_info:
                        movies_data.append(movie_info)
            else:
                # 通常の映画情報を処理
                movie_info = process_movie_box(box)
                if movie_info:
                    movies_data.append(movie_info)

        except Exception as e:
            print(f"Error processing movie: {e}")

    # DataFrameを作成
    df = pd.DataFrame(movies_data)

    # カラムの順序を整理
    columns = [
        "タイトル",
        "特集名",
        "制作年/国/時間",
        "監督/出演",
        "概要",
        "上映スケジュール",
        "料金",
        "受賞歴",
        "イベント情報",
    ]
    df = df.reindex(columns=columns)

    return df


def process_movie_box(box, is_special=False, special_title=None):
    # 基本情報を取得
    title_elem = box.find("span", class_="eiga-title")
    stuff_elem = box.find(["p", "span"], class_="stuff")
    note_elem = box.find(["p", "span"], class_="note")
    day_elem = box.find(["p", "span"], class_="day")
    price_elem = box.find("p", class_="price")
    award_elem = box.find("p", class_="syo")
    event_elem = box.find("p", class_="p3")

    # タイトルがない場合はスキップ
    if not title_elem:
        return None

    # 基本情報を整理
    basic_info = ""
    director_cast = ""
    if stuff_elem:
        stuff_text = stuff_elem.text.strip()
        stuff_lines = stuff_text.split("\n")
        basic_info = clean_text(stuff_lines[0]) if stuff_lines else ""
        director_cast = (
            clean_text("\n".join(stuff_lines[1:])) if len(stuff_lines) > 1 else ""
        )

    return {
        "タイトル": clean_text(title_elem.text),
        "特集名": clean_text(special_title) if is_special else "",
        "制作年/国/時間": basic_info,
        "監督/出演": director_cast,
        "概要": clean_text(note_elem.text) if note_elem else "",
        "上映スケジュール": clean_text(day_elem.text) if day_elem else "",
        "料金": clean_text(price_elem.text) if price_elem else "",
        "受賞歴": clean_text(award_elem.text) if award_elem else "",
        "イベント情報": clean_text(event_elem.text) if event_elem else "",
    }


def process_html_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        html_content = file.read()

    return scrape_movie_info(html_content)

In [12]:
html_content = "https://shimotakaidocinema.com/schedule/schedule.html"

df = scrape_movie_info(html_content)
df.head()

,タイトル,制作年/国/時間,概要,詳細情報,上映スケジュール,料金,受賞歴
0,リュミエール,,,,,,
1,思春期,,,,,,
2,リリアン・ギッシュの肖像,,,,,,
3,ジョイランド わたしの願い,2022年/パキスタン/2h07,パキスタンの大都市ラホール。トランスジェンダー女性のダンサーに恋する夫、働き続けたい妻。若き...,2022年/パキスタン/2h07\n\t\t監督・脚本：サーイム・サーディク\n\t\t出演...,2/8(土)～2/14(金) 12：40～(終14：50),一般1700円 / 大学・専門1300円 / シニア1100円 / 小・中・高1000円 /...,カンヌ国際映画祭「ある視点」部門審査員賞・クィア・パルム賞
4,ルート29,2024年/日本/2h00,清掃員・のり子は仕事先の入院患者から娘ハルを連れてきてほしいと頼まれ…。姫路から鳥取まで１本...,2024年/日本/2h00\n\t\t監督・脚本：森井勇佑『こちらあみ子』\n\t\t原作：...,2/8(土)～2/14(金) 15：05～(終17：10),一般1700円 / 大学・専門1300円 / シニア1100円 / 小・中・高1000円 /...,


In [18]:
print(df.head(10).to_markdown())

|    | タイトル                                 | 制作年/国/時間                  | 概要                                                                                                                                                       | 詳細情報                                                                               | 上映スケジュール                      | 料金                                                                        | 受賞歴                                                           |
|---:|:-----------------------------------------|:--------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------|:--------------------------------------|:----------------------------------------------------------------------------|:-----------------------------------------------------------------|